# Simple Plotting


In [ ]:
RESULTS_PATH = "../../your_sweep_path/default"

PLOT_ALL_SEEDS = False
# Full sweep
MODELS_TO_PLOT = ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", "Qwen/Qwen-1_8B", "Qwen/Qwen-7B", "Qwen/Qwen-14B"]
# Minimal sweep
# MODELS_TO_PLOT = ["gpt2", "gpt2-medium", "gpt2-large"]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from IPython.display import display

import os
import glob
import json

In [ ]:
records = []
for result_filename in glob.glob(os.path.join(RESULTS_PATH, "**/results_summary.json"), recursive=True):
    config_file = os.path.join("/".join(result_filename.split("/")[:-1]), "config.json")
    config = json.load(open(config_file, "r"))
    if config["model_size"] not in MODELS_TO_PLOT:
        continue
    if 'seed' not in config:
        config['seed'] = 0
    record = config.copy()
    if 'weak_model' in config:
        for k in record['weak_model']:
            if k == 'model_size':
                assert record['weak_model'][k] == record['weak_model_size']
            record['weak_' + k] = record['weak_model'][k]
        del record['weak_model']
    record.update(json.load(open(result_filename)))
    records.append(record)

df = pd.DataFrame.from_records(records).sort_values(['ds_name', 'model_size'])

In [ ]:
datasets = df.ds_name.unique()
for dataset in datasets:
    cur_df = df[(df.ds_name == dataset)].copy()
    base_accuracies = cur_df[cur_df['weak_model_size'].isna()].groupby('model_size').agg({'accuracy': 'mean', 'seed': 'count'}).sort_values('accuracy')
    base_accuracy_lookup = base_accuracies['accuracy'].to_dict()
    base_accuracies = base_accuracies.reset_index()

    cur_df['strong_model_accuracy'] = cur_df['model_size'].apply(lambda x: base_accuracy_lookup[x])
    cur_df.loc[~cur_df['weak_model_size'].isna(), 'weak_model_accuracy'] = cur_df.loc[~cur_df['weak_model_size'].isna(), 'weak_model_size'].apply(lambda x: base_accuracy_lookup[x])

    # Exclude cases where the weak model is better than the strong model from PGR calculation.
    valid_pgr_index = (
        (~cur_df['weak_model_size'].isna()) & 
        (cur_df['weak_model_size'] != cur_df['model_size']) & 
        (cur_df['strong_model_accuracy'] > cur_df['weak_model_accuracy'])
    )
    cur_df.loc[valid_pgr_index, 'pgr'] = (cur_df.loc[valid_pgr_index, 'accuracy'] - cur_df.loc[valid_pgr_index, 'weak_model_accuracy']) / (cur_df.loc[valid_pgr_index, 'strong_model_accuracy'] - cur_df.loc[valid_pgr_index, 'weak_model_accuracy'])

    cur_df.loc[cur_df['weak_model_size'].isna(), "weak_model_size"] = "ground truth"

    for seed in [None] + (sorted(cur_df['seed'].unique().tolist()) if PLOT_ALL_SEEDS else []):
        plot_df = cur_df.copy().sort_values(['strong_model_accuracy']).sort_values(['loss'], ascending=False)
        if seed is not None:
            plot_df = plot_df[plot_df['seed'] == seed]

        print(f"Dataset: {dataset} (seed: {seed})")

        pgr_results = plot_df[~plot_df['pgr'].isna()].groupby(['loss']).aggregate({"pgr": "median"})

        palette = sns.color_palette('colorblind', n_colors=len(plot_df['weak_model_size'].unique()) - 1)
        color_dict = {model: ("black" if model == 'ground truth' else palette.pop()) for model in plot_df['weak_model_size'].unique()}

        sns.lineplot(data=plot_df, x='strong_model_accuracy', y='accuracy', hue='weak_model_size', style='loss', markers=True, palette=color_dict)
        pd.plotting.table(plt.gca(), pgr_results.round(4), loc='lower right', colWidths=[0.1, 0.1], cellLoc='center', rowLoc='center')
        plt.xticks(ticks=base_accuracies['accuracy'], labels=[f"{e} ({base_accuracy_lookup[e]:.4f})" for e in base_accuracies['model_size']], rotation=90)
        plt.title(f"Dataset: {dataset} (seed: {seed})")
        plt.legend(loc='upper left')
        suffix = ""
        if seed is not None:
            suffix = f"_{seed}"
        plt.savefig(f"{dataset.replace('/', '-')}{suffix}.png", dpi=300, bbox_inches='tight')
        plt.show()

# Calibration w.r.t. the soft labels

In [ ]:
import os
from datasets import load_from_disk
import numpy as np

# strong_path = "../results/conf_disagree/bs=32-dn=sciq_with_supp-ge=3-lp=0-l=xent-l=4e-05-ls=cosi_anne-mc=512-ms=Qwen1.5-0.5B-ntd=500-ntd=4300-ntd=4300-o=adam-s=0-twd=0"
# w2s_path = "../results/conf_disagree/bs=32-dn=sciq_with_supp-lp=0-l=kl-l=2e-05-ls=cosi_anne-mc=512-ms=Qwen1.5-0.5B-ntd=500-ntd=4300-ntd=4300-o=adam-s=0-see=15-twd=0-we=2-wlf=0.5-wms=opt-350m"
strong_path = "../results/grads/bs=32-dl=1-dn=sciq_for_lm_head_with_supp-gib=1-ge=3-gee=10000000-lp=0-lbmae=0-l=xent-l=4e-05-ls=cosi_anne-mc=512-mfbm=auro_agai_supe-ms=Qwen1.5-0.5B-ntd=1000-ntd=3000-ntd=6000-o=adam-stl=50-s=0-twd=0"
w2s_path = "../results/grads/bs=32-dl=1-dn=sciq_for_lm_head_with_supp-gib=1-lp=0-lbmae=0-l=kl-l=2e-05-ls=cosi_anne-mc=512-mfbm=auro_agai_supe-ms=Qwen1.5-0.5B-ntd=1000-ntd=3000-ntd=6000-o=adam-stl=50-s=0-twd=0-we=3-wee=25-wlf=0.5-wms=opt-350m"
eval_results_paths = [p for p in os.listdir(w2s_path) if p.startswith("eval_results") and p[-1].isdigit()]
eval_results_paths.sort(key=lambda x: int(x.split("_")[-1]))

weak_soft_labels = None
gt_labels = None
pred_probs = []
for p in eval_results_paths:
    eval_results = load_from_disk(os.path.join(w2s_path, p)).with_format("numpy")
    if weak_soft_labels is None:
        weak_soft_labels = eval_results['weak_soft_label'][:, 1]  # type: ignore
    else:
        assert np.all(weak_soft_labels == eval_results['weak_soft_label'][:, 1])  # type: ignore
    if gt_labels is None:
        gt_labels = eval_results['soft_label'][:, 1]  # type: ignore
    else:
        assert np.all(gt_labels == eval_results['soft_label'][:, 1])  # type: ignore
    pred_probs.append(eval_results['soft_pred'][:, 1])  # type: ignore

weak_soft_labels = np.array(weak_soft_labels)
pred_probs = np.array(pred_probs)
gt_labels = np.array(gt_labels)
weak_soft_labels.shape, pred_probs.shape, gt_labels.shape

In [ ]:
import plotly.graph_objs as go
import numpy as np

n_time_steps = len(pred_probs)
# Create base figure
fig = go.Figure(
    data=[go.Scatter(x=pred_probs[0], y=weak_soft_labels, mode='markers',  
                     marker=dict(color=gt_labels, colorscale='Viridis', colorbar=dict(title='GT Label')))],
    layout=go.Layout(
        title="Soft Labels Over Time",
        xaxis=dict(range=[0, 1], title="Strong student predicted probability"),
        yaxis=dict(range=[0, 1], title="Weak supervisor soft label"),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 700, "redraw": True}, "fromcurrent": True}]),
                    dict(label="Pause",
                         method="animate",
                         args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}])
                    ])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=pred_probs[i], y=weak_soft_labels, mode='markers',
                                        marker=dict(color=gt_labels, colorscale='Viridis', colorbar=dict(title='GT Label')))],
                         name=str(i))
            for i in range(1, n_time_steps)]
)

# Add axis titles
fig.update_layout(xaxis_title="Strong student predicted probability", yaxis_title="Weak supervisor soft label", height=800, width=900)


# Show animation
fig.show()

# Get gradients per example

In [ ]:
strong_eval_results_path = os.path.join(strong_path, "eval_results_final")
final_eval_results_path = os.path.join(w2s_path, "eval_results_final")
final_eval_results = load_from_disk(final_eval_results_path)
strong_eval_results = load_from_disk(strong_eval_results_path)
print(len(final_eval_results), len(strong_eval_results))
strong_eval_results = strong_eval_results.select(range(len(final_eval_results)))
assert np.all(np.array(final_eval_results['id']) == np.array(strong_eval_results['id']))
final_eval_results = final_eval_results.add_column("strong_soft_pred", strong_eval_results['soft_pred'])  # type: ignore
final_eval_results = final_eval_results.add_column("strong_hard_pred", strong_eval_results['hard_pred'])  # type: ignore
final_eval_results = final_eval_results.with_format("torch")

In [ ]:
import json
from tqdm.auto import tqdm

import torch
from collections import defaultdict

from weak_to_strong.datasets import load_and_process_dataset, tokenize_dataset
from weak_to_strong.config import ModelConfig, MODELS_DICT, LOSS_DICT
from weak_to_strong.model import TransformerWithHead
from weak_to_strong.common import get_tokenizer, to_batch
from weak_to_strong.train import maybe_load_model

config = json.load(open(os.path.join(w2s_path, "config.json"), "r"))
model_name = config["model_size"]

In [ ]:
use_lm_head = "choice_input_ids" in final_eval_results.column_names
loss_fn = LOSS_DICT[config["loss"]]

d_proj = 10_000
proj_grads, hiddens = defaultdict(lambda: defaultdict(dict)), defaultdict(lambda: defaultdict(dict))

In [ ]:
for run_type, results_path in [("w2s", w2s_path), ("strong", strong_path),]:

    # init model
    mcfg = MODELS_DICT[model_name].copy()
    if config["disable_lora"]:
        mcfg["lora_modules"] = None
    model_config = ModelConfig(**mcfg)
    model = TransformerWithHead.from_pretrained(  # type: ignore
                model_config.name,
                lora_modules=model_config.lora_modules,
                use_lm_head=use_lm_head,
                num_labels=2,
                linear_probe=config["linear_probe"],
                **model_config.custom_kwargs,
            ).cuda()

    model_n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    torch.manual_seed(0)  # ensure that our projection is the same across runs and examples
    proj_basis_indices = torch.randint(0, model_n_params, (d_proj,))
    print(f"Hash(projection indices): {proj_basis_indices.sum().item()}")
    
    for target_label_column in ["weak_soft_label", "soft_label"]:
        print(f"Computing gradients for {target_label_column} in {run_type}")
        for checkpoint in os.listdir(results_path):
            if not checkpoint.startswith("checkpoint") or run_type in proj_grads and checkpoint in proj_grads[run_type] and target_label_column in proj_grads[run_type][checkpoint]:
                continue
            print(f"Loading model from {checkpoint}")
            proj_grads[run_type][checkpoint][target_label_column] = torch.zeros((len(final_eval_results), d_proj), device=model.device)
            hiddens[run_type][checkpoint][target_label_column] = torch.zeros((len(final_eval_results), model.config.hidden_size), device=model.device)

            # load model checkpoint
            assert maybe_load_model(model, os.path.join(results_path, checkpoint))
            model.eval()

            for i, batch in tqdm(enumerate(to_batch(final_eval_results, batch_size=1))):
                input_ids = torch.nn.utils.rnn.pad_sequence(
                            [ex for ex in batch["input_ids"]], batch_first=True
                    ).to(model.device)
                label = batch[target_label_column].to(model.device)
                        
                logits, hs = model(
                                input_ids, choice_input_ids=batch.get("choice_input_ids"), output_hidden_states=True
                    )
                    
                loss = loss_fn(logits, label, step_frac=0)
                loss.backward()

                # get grad in a vector
                grad = torch.cat([p.grad.clone().detach().flatten() for p in model.parameters() if p.requires_grad])
                # project onto random direction
                proj_grads[run_type][checkpoint][target_label_column][i, :] = grad[proj_basis_indices]
                # proj_grads[i] = torch.cat([p.grad.clone().detach().flatten() for p in model.score.parameters() if p.requires_grad])

                hiddens[run_type][checkpoint][target_label_column][i, :] = hs[-1][0, -1, :].detach().clone().cpu()

                # zero out grads
                model.zero_grad()

In [ ]:
def to_dict(defdict):
    return {k: to_dict(v) if isinstance(v, defaultdict) else v for k, v in defdict.items()}
torch.save(to_dict(proj_grads), os.path.join(w2s_path, "proj_grads.pt"))
torch.save(to_dict(hiddens), os.path.join(w2s_path, "hiddens.pt"))

In [ ]:
set(proj_grads["w2s"].keys()) - set(proj_grads["strong"].keys())
ckpt = "checkpoint_0.bin"
target_label_column = "soft_label"
ckpt_grads = proj_grads["w2s"][ckpt][target_label_column]
# ckpt_hiddens = hiddens["w2s"][ckpt]

In [ ]:
proj_grads_normalized = ckpt_grads / ckpt_grads.norm(dim=1, keepdim=True)

In [ ]:
def get_answer_str(txt):
    last_quote = txt.rfind('"')
    penultimate_quote = txt.rfind('"', 0, last_quote)
    return txt[penultimate_quote + 1:last_quote]
is_answer_in_support = np.array([s.count(get_answer_str(s)) > 1 for s in final_eval_results["txt"]]) 
exists_support = np.array([not s.startswith("Name: Bob\n\nPassage 1:\n\n\nQ1:") for s in final_eval_results["txt"]])

In [ ]:
soft_labels = np.array(final_eval_results["soft_label"])[:, 1]
weak_soft_labels = np.array(final_eval_results["weak_soft_label"])[:, 1]
strong_soft_preds = np.array(final_eval_results["strong_soft_pred"])[:, 1]
w2s_soft_preds = np.array(final_eval_results["soft_pred"])[:, 1]

weak_error = (weak_soft_labels - soft_labels)
weak_diff_from_strong = (weak_soft_labels - strong_soft_preds)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
sns.set_style('whitegrid')

# project gradients into 2D

use_normalized = False
mat = proj_grads_normalized if use_normalized else ckpt_grads

pca = PCA(n_components=10)
proj_2d = pca.fit_transform(mat.cpu().numpy())
# proj_2d = pca.transform(mat.cpu().numpy())
print("explained variance", pca.explained_variance_ratio_)
plt.figure(figsize=(4, 3), dpi=250)
plt.scatter(proj_2d[:, 0], proj_2d[:, 1], c=weak_error, cmap='coolwarm', alpha=0.5, marker='o', vmin=-1, vmax=1)
plt.colorbar()
plt.xlabel("Gradient PC 1")
plt.ylabel("Gradient PC 2")
plt.title("Normalized gradients projected into 2D" if use_normalized else "Gradients projected into 2D")
plt.show()